In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import wandb
import torch
import torch.nn as nn

import utils
from naive_model import NaiveModel
import encoding

In [266]:
NUM_EMBEDDING = 2000

assert torch.cuda.is_available()
torch.backends.cudnn.benchmark = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

class Model(nn.Module):
    arch = 'attent-LEE-mixlh1'
    def __init__(self, units, combinations=encoding.NAMES):
        super().__init__()
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        
        self.pre_lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=False, bidirectional=True)
        
        self.attention = nn.MultiheadAttention(units, num_heads=1)
        
        self.post_lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=False, bidirectional=True)

        self.tasks = {}
        for combination in combinations:
            out = nn.Linear(in_features=units, out_features=encoding.class_size(combination))
            self.tasks[combination] = out
            setattr(self, encoding.class_name(combination), out)

    def isroot(self, combination):
        return any(r in combination for r in ['R1', 'R2', 'R3', 'R4'])
        
    def forward(self, x):
        embeds = self.embed(x).permute([1, 0, 2])
        
        lstm_out, (h_n0, c_n) = self.pre_lstm(embeds)
        left, right = torch.chunk(lstm_out, 2, dim=-1)
        lstm_out = torch.squeeze(left + right)

        attention, weights = self.attention(lstm_out, embeds, embeds)
        
        lstm_out, (h_n1, c_n) = self.post_lstm(attention + embeds + lstm_out)
        
        h_n = h_n0 + h_n1

        return {combination: f(h_n[0] + h_n[1])
                for combination, f in self.tasks.items()}


In [265]:
def sanity():
    model = to_device(Model(100, combinations=[('B', 'T')]))
    print(model)
    with torch.no_grad():
        verbs = encoding.wordlist2numpy(["אתאקלם", "יכפיל", "בואס"])
        labels = {'B': torch.Tensor([3, 5]), 'T': torch.Tensor([2, 4])}
        verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
        tag_scores = model(verbs)
        print(f'{tag_scores[("B", "T")].shape=}')
        for combination in tag_scores:
            print(combination)
            v = tag_scores[combination]
            # print(f'{v=}')
            print(f'{labels=}')
            print()
sanity()

Model(
  (embed): Embedding(2000, 100)
  (pre_lstm): LSTM(100, 100, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): Linear(in_features=100, out_features=100, bias=True)
  )
  (post_lstm): LSTM(100, 100, bidirectional=True)
  (BxT): Linear(in_features=100, out_features=28, bias=True)
)
tag_scores[("B", "T")].shape=torch.Size([3, 28])
('B', 'T')
labels={'B': tensor([3., 5.]), 'T': tensor([2., 4.])}



In [194]:

# TEMP_PATH = 'model.pt'

#                 best_lr = 8e-4
#                 best_loss = 10
                
#                 torch.save({
#                     'state_dict': model.state_dict(),
#                     'optimizer': optimizer.state_dict(),
#                 }, TEMP_PATH)
                
#                 for i in range(1):
#                     checkpoint = torch.load(TEMP_PATH)
#                     model.load_state_dict(checkpoint['state_dict'])
#                     optimizer.load_state_dict(checkpoint['optimizer'])

def fit(model, train, test, *, epochs,  runsize, criterion, optimizer, phases, teacher, batch_size):
    x_train, y_train = utils.batch_xy(train, batch_size)
    x_test, y_test = utils.batch_xy(test, batch_size)
    x = {'train': x_train, 'test': x_test}
    y = {'train': y_train, 'test': y_test}

    stats = utils.Stats(model.tasks.keys(), runsize)
    
    for epoch in range(epochs):
        stats.epoch_start()
        
        for phase in phases:
            model.train(mode=phase == 'train')

            stats.phase_start(phase, batches_in_phase=len(x[phase]))

            for inputs, labels in zip(x[phase], y[phase]):
                stats.batch_start()

                inputs = to_device(inputs)
                labels = to_device(labels)

                with utils.conditional_grad(phase=='train'):
                    outputs = model(inputs)

                if teacher is not None:
                    pseudo_labels = teacher(inputs)
                    losses = {k: criterion(outputs[k].double(), pseudo_labels[k]) for k in outputs}
                else:
                    losses = {combination: criterion(output.double(), labels[combination])
                              for combination, output in outputs.items()}

                if phase == 'train':
                    stats.assert_reasonable_initial(losses, nn.CrossEntropyLoss)

                loss = sum(losses.values())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                stats.update(loss=loss.item(),
                             batch_size=inputs.size(0),
                             d={k: (outputs[k], labels[k]) for k in outputs})
                
                stats.batch_end()
            stats.phase_end()
        stats.epoch_end()
    return stats

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = encoding.wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {}
    # FIX: assumes no overlaps
    for combination, v in outputs.items():
        combined_index = torch.argmax(v).cpu().data.numpy()
        indices = np.unravel_index(combined_index, encoding.combined_shape(combination))
        if isinstance(combination, str):
            combination = tuple([combination])
        for k, i in zip(combination, indices):
            # assert k not in res, "Overlapping classes are not handled"
            s = k
            if k in res:
                s += "'"
            res[s] = encoding.from_category(k, i)
    if all(r in res for r in ['R1', 'R2', 'R3', 'R4']):
        res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res

In [14]:
torch.manual_seed(0)
np.random.seed(0)

arity = '3'
gen = 'all'
artifact_name = f'{gen}_{arity}_shufroot'
filename = f'synthetic/{artifact_name}.tsv'  # all_verbs_shuffled
test_size = 8192

artifact = wandb.Artifact(artifact_name, type='dataset')
artifact.add_file(filename)

(train_x, pre_train_y), (test_x, pre_test_y) = encoding.load_dataset_split(filename, split=test_size)

utils.shuffle_in_unison([train_x, *pre_train_y.values()])

In [9]:
def naive_config(filename):
    return {
        'model': NaiveModel.learn_from_file(filename),
        'phases': ['test'],
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': None
    }

def standard_config(model, lr):
    return {
        'model': model,
        'criterion': nn.CrossEntropyLoss(),
        'phases': ['train', 'test'],
        'teacher': None
    }

def teacher_config(train, model, lr):
    res = standard_config(model, lr)
    res['teacher'] = NaiveModel.learn_from_data(train)
    res['criterion'] = nn.BCEWithLogitsLoss()  # BCELoss: works, but total loss is nan
    return res

In [267]:
%env WANDB_SILENT true
%env WANDB_MODE run

config = {
    'epochs': 1,
    'test_size': test_size,
}

def experiment(units, lr, batch_size):
    torch.manual_seed(1)
    np.random.seed(1)
    
    combinations = list(encoding.CLASSES)
    
    train_y = utils.ravel_multi_index(pre_train_y, combinations)
    test_y = utils.ravel_multi_index(pre_test_y, combinations)
    train = (train_x, train_y)
    test = (test_x, test_y)
    
    config.update({
        'runsize': 2 * 8192 // batch_size,
        'batch_size': batch_size,
        'units': units,
        'lr': lr,
        'optimizer': 'adam'
    })
    
    model = to_device(Model(units=units, combinations=combinations))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    names_str = '+'.join(encoding.class_name(combination) for combination in combinations)
    run = wandb.init(project="rootem",
                     group=f'attention',  # f'lr_units_grid_search-{arity}-{wandb.util.generate_id()}',
                     name=f'{model.arch}-{units}-{lr:.0e}-{batch_size}',  # f'{gen}-{arity}-{lr:.0e}',# f'{arity}-batch_{BATCH_SIZE}', # f'all-{arity}-lr_{lr:.0e}-units_{units}',
                     tags=[gen, arity, 'synthetic', 'shuffle-root', 'no_prefix', 'shuffle'],
                     config=config)
    with run:
        run.use_artifact(artifact)

        wandb.config.update(config, allow_val_change=True)

        print(config)

        if isinstance(model, nn.Module):
            wandb.watch(model)

        stats = fit(train=train,
            test=test,
            epochs=config['epochs'],
            runsize=config['runsize'],
            optimizer=optimizer,
            batch_size=batch_size,
            **standard_config(model, lr)
        )
        wandb.save(f"simple_{arity}.h5")

    return model, stats

units = 200
batch_size = 128
lr = 8e-4
model, stats = experiment(units, lr, batch_size)


env: WANDB_SILENT=true
env: WANDB_MODE=run


{'epochs': 1, 'test_size': 8192, 'runsize': 128, 'batch_size': 128, 'units': 200, 'lr': 0.0008, 'optimizer': 'adam'}
 1  5376/ 5445 B_acc: 0.826 T_acc: 0.933 V_acc: 0.696 G_acc: 0.790 P_acc: 0.980 R1_acc: 0.973 R2_acc: 0.815 R3_acc: 1.000 R4_acc: 0.984 Loss: 1.9079
 1    64/   64 B_acc: 0.829 T_acc: 0.914 V_acc: 0.699 G_acc: 0.802 P_acc: 0.978 R1_acc: 0.925 R2_acc: 0.869 R3_acc: 1.000 R4_acc: 0.943 Loss: 2.2270


In [ ]:
def print_words(model):
    print(predict(model, 'הבריל'))
    print(predict(model, 'חגוו'))
    print(predict(model, 'עגו'))
    print(predict(model, 'צירלל'))
    print(predict(model, "השטקרפתי"))
    print(predict(model, "ישסו"))
print_words(model)

In [ ]:
input("$ ")
input("$ ")

In [ ]:
wandb.join()

NameError: name 'BATCH_SIZE' is not defined